# Стохастический анализ (2024)
# Домашнее Задание 4

ВАШЕ ИМЯ ЗДЕСЬ

Оценка: ?? из 100


Задачи: ?? из 

Ноутбук: ?? из 40


Дедлайн: ??, 2359МСК

Решённый ноутбук нужно засабмитить в класрум курса

Пожалуйста, называйте файл(ы) в формате <Имя>HW4.pdf или <Имя>HW4.ipynb. Например, KaledinHW4.pdf и KaledinHW4.ipynb.

In [ ]:
#all imports here, pls
import numpy as np
import scipy.stats as spstats

import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

В этом ноутбуке мы более плотно поисследуем некоторые физические явления в микро- и макромасштабе с помощью численных методов, а также ещё посмотрим на MCMC в действии.

## Диффузии (30 баллов)


### (20 баллов) Алгоритмы семплирования

Мы будем исследовать две цепи: от алгоритма ULA и алгоритма MALA. Для реализации этих методов вам прежде всего понадобится метод Эйлера (используйте свою реализацию или дополните реализацию ниже).

In [ ]:
class SDESolver:
    
    def __init__(self, b, sigma):
        self.b = b 
        self.sigma = sigma
    
    def step(self, x, t, tnext):
        """One integration step

        Raises:
            NotImplementedError: if not implemented
        """        
        raise NotImplementedError
    
    #added returnLast to not store everything
    #added kwargs to send additional arguments to step through solve
    def solve(self, x0, ts, returnLast=False, loaderLabel="Solving....", **kwargs):
        """Computes the solution on the given grid ts with x0 as initial condition

        Args:
            x0 float[]: initial condition (N,d,) or (d,)
            ts float[]: time grid (T,)
            tIds float[]: optional, ids in the dense time grid (sent through kwargs, needed for experiments)
        Returns:
            xs float[]: approximated solution (N,d,T) at points ts
        """
        try:
            if(len(x0.shape)==1):
                x0 = x0[None, :]
        except:
            #a number?
            x0 = np.ones([1,1])*x0
            
        
        if(returnLast):
            xs = x0    
        else:            
            xs = np.zeros([x0.shape[0], x0.shape[1], len(ts)])
            xs[...,0] = x0
        
        print(loaderLabel)
        for i in tqdm.tqdm(np.arange(1,len(ts))):
            if(returnLast):
                xs = self.step(xs,ts[i-1],ts[i],**kwargs)
            else:
                xs[...,i] = self.step(xs[...,i-1],ts[i-1],ts[i],**kwargs)
        
        return xs
    
class EulerSolver(SDESolver):
    
    def __init__(self,b,sigma):
        super(EulerSolver, self).__init__(b,sigma)
        
    def step(self, x, t, tnext,**kwargs):
        """Makes Euler step

        Args:
            x float[]: previous point (N,d)
            t float: previous time
            tnext float: next time
        """            
        #b (N,d)  x (N,d)   sigma (N,d,d)
        #NEW!! improved to handle sigma as matrix
        if(x.ndim==1): #in case we still got (d,)
            x = x[None,:]
        
        #???YOUR CODE
       
        #res = ???
        #return res
        pass

Ещё нам понадобится обёртка для семплера, семплер ULA и семплер MALA.

In [ ]:
class Sampler:
    
    def __init__(self):
        pass
    
    def sample(self, N):
        """Samles N points

        Args:
            N (int): number of points to sample

        Raises:
            NotImplementedError: since needs override
        """        
        raise NotImplementedError

#for simplicity, a start sampler
class ConstSampler(Sampler):
    
    def __init__(self, const):
        '''
        Returns a const vector const (d,)
        '''
        self.const = const
        
    def sample(self,N):
        return self.const[None,:]*np.ones([N,1])
    
    
class GaussianSampler(Sampler):
    
    def __init__(self, mu, cov):
        self.mu = mu
        self.cov = cov
    
    def sample(self,N):
        return np.random.multivariate_normal(self.mu,self.cov, size=(N,))
    
class UniformSampler(Sampler):
    
    def __init__(self, left, right):
        self.left = left
        self.right = right
    
    def sample(self,N):
        return np.random.uniform(size=(N,self.left.shape[0]))*(self.right - self.left) + (self.right + self.left)/2
    
    
class ULASampler(Sampler):
    
    def __init__(self, gradLog, startSampler, delta=0.01):
        """
        Args:
            gradLog functionHandler: function mapping R^d tp R^d, gradient of the logarithm
            delta float: discrete time step, defaults to 0.01
            startSampler functionHandler: maps int N to a set of N samples
        """        
        super(ULASampler,self).__init__()
        self.gradLog = gradLog
        self.delta = delta
        self.startSampler = startSampler
        
        #pass t because of solver
        def sigg(t,x):
            return np.sqrt(2)*np.eye(x.shape[1])[None,...] * np.ones([x.shape[0],x.shape[1],x.shape[1]])
        self.solver = EulerSolver(self.gradLog,sigma=sigg)
        
    def sample(self, N, burnIn=20000, parallelChains=False):
        """Samles N points

        Args:
            N (int): number of points to sample
            burnIn (int): number of samples to burn before sampling

        Raises:
            NotImplementedError: since needs override
        """        
        Tburned = burnIn*self.delta
        tsBurn = np.arange(0,Tburned+3*self.delta/2,self.delta)#if you need it....
        if(parallelChains):
            #CASE of parallel sampling: run N chains in parallel, collect the last state
            startPoints = self.startSampler.sample(N)
            #make burn-in, save the last observation
            #result = #???
            print("Sampling...")
            #actually the last observation is the sample
        else:
            #CASE of sequential sampling: wait and then collect, one trajectory
            startPoints = self.startSampler.sample(1)
            #make burn-in, save the last observation
            #then collect samples
            #burned = ???
            result = np.zeros([N,startPoints.shape[-1]])
            #result[0,:]=burned
            print("Sampling...")
            for i in tqdm.tqdm(np.arange(1,N)):
                #????
            #result (N,d) is the sample of N observations
                
        #return result
        pass
    
    def _odeStep(self,x):
        #one step of the solver
        return self.solver.step(x,0,self.delta)
         
    

Проверим теперь ULASampler на простом примере семплирования из $N(1,1)$. Постройте диффузию Ланжевена для семплирования из этого распределения.

ВАШЕ РЕШЕНИЕ ТУТ 

In [ ]:
#pass t because of solver
def gradLog(t,x):
    return -(x-1)

startSampler = ConstSampler(np.zeros([1]))
delta = 0.001
ulaSampler = ULASampler(gradLog,startSampler,delta)

In [ ]:
sampleParallel = ulaSampler.sample(10000,burnIn=100000, parallelChains=True)
sampleSequential = ulaSampler.sample(10000,burnIn=100000, parallelChains=False)
print(sampleParallel.shape,sampleSequential.shape)

In [ ]:
f, ax = plt.subplots(figsize=(5,5))

ax.grid()
ax.set_title("Гистограммы семплов")
ax.hist(sampleParallel[:,0], alpha=0.7,density=True,)
ax.hist(sampleSequential[:,0], alpha=0.7,density=True)
ax.legend(["Parallel","Sequential"])

xx = np.arange(-3,4,0.01)
normPDF = spstats.norm.pdf(xx-1)
ax.plot(xx,normPDF)

У Sequential-версии распределение немного отличается в силу коррелированности семплов, но у обеих версий примерно в одном месте находятся мода и матожидание.

Теперь алгоритм MALA. Он несильно отличается от ULA, но нам понадобится дополнительно знание плотности с точностью до константы нормировки.

Ещё нужно явно выписать пропоузал, он НЕсимметричный и его надо вычислять. Нужно с точностью до нормировочной константы(она сократится) вычислить

$$
q(X' \vert X_t) = ???
$$

$$
q(X_t \vert X') = ???
$$

Запишите формулы для плотностей пропоузала в ULA ниже (это важно! потом может быть очень сложно искать ошибку).

ВАШЕ РЕШЕНИЕ

In [ ]:
class MALASampler(ULASampler):
    
    def __init__(self, gradLog, startSampler, density, delta=0.01):
        """
        Args:
            gradLog functionHandler: function mapping R^d tp R^d, gradient of the logarithm
            delta float: discrete time step, defaults to 0.01
            startSampler functionHandler: maps int N to a set of N samples
            density functionHandler: maps R^d tp R, density function up to normalization constant
        """        
        super(MALASampler,self).__init__(gradLog, startSampler, delta)
        self.density = density
        
    def sample(self, N, burnIn=20000, parallelChains=False):
        """Samles N points

        Args:
            N (int): number of points to sample
            burnIn (int): number of samples to burn before sampling

        Raises:
            NotImplementedError: since needs override
        """        

        if(parallelChains):
            #CASE of parallel sampling: run N chains in parallel, collect the last state
            startPoints = self.startSampler.sample(N)
            print("Burning...")
            res = startPoints
            for i in tqdm.tqdm(np.arange(burnIn)):
                #??? YOUR CODE
                pass
            print("Sampling...")
            result = res
        else:
            #CASE of sequential sampling: wait and then collect, one trajectory
            startPoints = self.startSampler.sample(1)
            print("Burning...")
            #make burn in with _MHStep, save the last state
            #then collect the samples
            result = np.zeros([N,startPoints.shape[-1]])
            result[0,:]=res
            print("Sampling...")
            for i in tqdm.tqdm(np.arange(1,N)):
                #?? YOUR CODE
                pass
                
        #return result
    
    def _odeStep(self,x):
        return self.solver.step(x,0,self.delta)
    
    def _MHStep(self, x):
        
        xNew = # make a proposal with _odeStep(x) #t is bypassed
        
        #calculate proposal log densities 
        #x->xNew
        #xxNewLogDens = ???
        #xNew->x
        #xNewxLogDens = ???
        
        #calculate acceptance probability
        #use np.log WHEREVER YOU CAN for numerical stability
        accRatios = #????
        acceptProbs = #????
        accepted = np.random.uniform(size=acceptProbs.shape)<acceptProbs
        
        #make a decision
        return accepted[:,None]*xNew + (1-accepted)[:,None]*x

        
        

In [ ]:
#pass t because of solver
def gradLog(t,x):
    return -(x-1)
def density(x):
    return np.exp(-(x-1)**2/2).squeeze()

startSampler = ConstSampler(np.zeros([1]))
delta = 0.01
malaSampler = MALASampler(gradLog,startSampler,density,delta)

In [ ]:
sampleParallel = malaSampler.sample(10000,burnIn=100000, parallelChains=True)
sampleSequential = malaSampler.sample(10000,burnIn=100000, parallelChains=False)
print(sampleParallel.shape,sampleSequential.shape)

In [ ]:
f, ax = plt.subplots(figsize=(5,5))

ax.grid()
ax.set_title("Гистограммы семплов(MALA)")
ax.hist(sampleParallel[:,0], alpha=0.7,density=True,)
ax.hist(sampleSequential[:,0], alpha=0.7,density=True)
ax.legend(["Parallel","Sequential"])

xx = np.arange(-3,4,0.01)
normPDF = spstats.norm.pdf(xx-1)
ax.plot(xx,normPDF)

Сходимость не зависит от того, с какого распределения мы стартовали. Давайте попробуем семплировать из такого же распределения, но запустим два варианта цепи (обе в режиме sequential): с началом в  $7$ и с началом в $-4$.

Нарисуйте ниже графики, где по оси х отложено время, а по оси y -- значения цепей. Не забуьте про $\texttt{burnIn}$.

In [ ]:
#pass t because of solver
def gradLog(t,x):
    return -(x-1)
def density(x):
    return np.exp(-(x-1)**2/2).squeeze()

startSampler1 = ConstSampler(np.array([7]))
startSampler2 = ConstSampler(np.array([-5]))
delta = 0.01
malaSampler1 = MALASampler(gradLog,startSampler1,density,delta)
malaSampler2 = MALASampler(gradLog,startSampler2,density,delta)
ulaSampler1 = ULASampler(gradLog,startSampler1,delta)
ulaSampler2 = ULASampler(gradLog,startSampler2,delta)

In [ ]:
experiments = {"MALA1": {"sampler":malaSampler1},
               "MALA2": {"sampler":malaSampler2},
               "ULA1": {"sampler":ulaSampler1},
               "ULA2": {"sampler":ulaSampler2}}
expResults = {expId: exp["sampler"].sample(10000,burnIn=1, parallelChains=False) for expId,exp in experiments.items()}

In [ ]:
f, ax = plt.subplots(figsize=(5,5))

ax.grid()
ax.set_title("Значения цепей")

expKeys = list(experiments.keys())
for key in expKeys:
    ax.scatter(np.arange(expResults[key].shape[0])[::20],expResults[key].squeeze()[::20],alpha=0.7)

ax.legend(expKeys)

MALA мало смысла запускать с плохого старта из региона с низкой плотностью, потому что мы часто будем отклонять семпл. Поэтому если алгоритм не сходится, всегда можно начать с ULA.

## Плотность посложнее (10 баллов)

Рассмотрим диффузию в $\mathbb{R}^2$

$$
dX_t = b(X_t)dt + \sqrt{2} dW_t, \quad b(x,y) = -0.5[x/250 + 4x(y+2x^2-10), ~ 2(y+2x^2-10) ]^T
$$

Вычислите стационарную плотность этой диффузии.

ВАШЕ РЕШЕНИЕ ЗДЕСЬ

### (5 баллов) Семплирование

Постройте семплы спомощью алгоритмов MALA и ULA из начального распределения, которое вам кажется удобнее, обоснуйте свой выбор.

Запустите параллельно $1000$ цепей для каждого алгоритма и нарисуйте на плоскости их положения в разные моменты времени $t=1,1000,10000,100000$(вам поможет реинициализация с помощью ConstExactSampler). Когда примерно цепь доходит до эргодического состояния? Попробуйте разные размеры шага $\texttt{delta}=0.01,0.1,0.001$, как изменятся результаты работы алгоритма? Для стартового разогрева можно использовать ULA, если в MALA много отвержений, а потом перейти на MALA.

In [ ]:
#pass t because of solver

def gradLog(t,x):
    #print(x.shape)
    #return ??? YOUR CODE
    pass
def logDensity(x):
    #return ??
    #YOUR CODE
    pass

startSampler = GaussianSampler(np.zeros([2])+4,np.array([[1,0],[0,1]]))
delta = 0.1
ulaSampler00 = ULASampler(gradLog,startSampler,delta)

In [ ]:
class ConstExactSampler(Sampler):
    
    def __init__(self, x):
        self.x = x
        
    def sample(self,N):
        return self.x
    
#dummy thing for checkpointing

In [ ]:
#samples = ???

In [ ]:
f, ax = plt.subplots(figsize=(7,7))

ax.grid()
ax.set_title("Значения цепей")

#example
#ax.scatter(samples00[:,0],samples00[:,1],alpha=0.7)


ВАШ КОД И КОММЕНТАРИИ